# Storing scraped articles

In [1]:
import numpy as np
import pandas as pd
import requests
import re
import sqlite3
import time
import json
import openai
import redis

In [2]:
r = redis.Redis("localhost", 5211, decode_responses=True)

In [3]:
db = sqlite3.connect("scraping.db") # 144139

In [4]:
cur = db.execute("select count(*) from article where headline = 'None'")
db.commit()
for row in cur:
    print(row)

(3824,)


In [5]:
n = 138955
while True:
    if (data := r.rpop("to_store")) is None:
        print(f"{n} Nothing to store, idling...", end="\r")
        time.sleep(3)
    else:
        data = json.loads(data)
        # print(data, end="\r")
        if len(data["headline"]) == 0:
            print("ERROR, NO HEADLINE")
            continue
        if len(data["content"]) == 0:
            print("ERROR, NO CONTENT")
            continue
        n += 1
        print(n, end="\r")
        db.execute(f'update article set headline = ?, content = ? where id = ?', (data["headline"], data["content"], int(data["id"])))
        db.commit()

KeyboardInterrupt: 